In [63]:
import fitz
from df2gspread import df2gspread as d2g
import pandas as pd
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from pandas.io.json import json_normalize
import re
from shutil import copyfile
test=True

In [64]:
%ls

In [65]:
%cp LDC-Draft1.pdf LDC-Draft1-annotated.pdf

In [66]:
### READ IN PDF
annotated_doc = fitz.open("LDC-Draft1-annotated.pdf")
original_doc = fitz.open("LDC-Draft1.pdf")

page_width = annotated_doc[0].rect.x1
print page_width

In [67]:
print "here"

In [68]:
base_url = "/comments/"
class Annotation:
    def __init__(self, timestamp, email, page_number, text, comment, code, areas, footnote_index=None, url=None):
        self.id = timestamp.replace(":","-").replace(".","-").lower()
        self.timestamp = timestamp
        self.email = email
        self.page_number = int(page_number)
        self.text = text
        self.comment = comment
        self.code = code
        self.areas = areas
        self.footnote_index = footnote_index
        self.footnote_location = (0,0)
        self.page_level = True
        self.url = "{}{}".format(base_url,self.id)
        self.chapter = None
        self.article = None
        self.division = None
    
    def __repr__(self):
        return "{} - {}".format(self.page_number, self.footnote_index)
    
    def find_note_location(self, doc):
        page = doc[self.page_number-1]
        text_instances = page.searchFor(self.text)

        ## Find the inst
        inst = False
        for inst in text_instances:
            inst
        if inst:
            self.footnote_location = (inst.x1, inst.y0)
            self.page_level = False
        return self.footnote_location
    
    def set_page_level_location(self):
        if self.page_level:
            x = 40 * self.footnote_index
            y = 5
            self.footnote_location = (x, y)
        return self.footnote_location
    
    def add_to_pdf(self, doc):
        """
        Adds the given annotation to the given PDF doc
        """
        page = doc[self.page_number-1]
        text_instances = page.searchFor(self.text)

        ## Add the highlight
        inst = False
        for inst in text_instances:
            highlight = page.addHighlightAnnot(inst)

        footnote_loc = fitz.Rect(self.footnote_location[0],self.footnote_location[1]-2,self.footnote_location[0]+10,self.footnote_location[1]+6)
        footnote = page.addFreetextAnnot(footnote_loc, str(self.footnote_index), fontsize=8)
        link = {'kind': 2, 
                'from': fitz.Rect(footnote_loc.x0-1, footnote_loc.y0-1, footnote_loc.x1+1, footnote_loc.y1+1),
                'type': 'uri',
                'uri': self.url}
        page.insertLink(link)
        if self.page_level:
            comment = page.addTextAnnot(
                (footnote_loc.x0+5,footnote_loc.y0), 
                "{}. {}".format(self.footnote_index, self.comment), 
                icon="Comment")
        else:
            if footnote_loc.x0 < page_width/2:
                comment = page.addTextAnnot(
                    (10,footnote_loc.y0), 
                    "{}. {}".format(self.footnote_index, self.comment), 
                    icon="Comment")
            else:
                comment = page.addTextAnnot(
                    (page_width-40,footnote_loc.y0), 
                    "{}. {}".format(self.footnote_index, self.comment), 
                    icon="Comment")
                
    def create_hugo_file(self, loc):
        """
        Creates a properly formatted Hugo file
        """
        f = open("{}{}.html".format(loc, self.id),"w+")
        f.write("---\n")
        if self.page_level:
            f.write('title: "Page {} - {}"\n'.format(self.page_number, self.footnote_index))
        else:
            f.write('title: "{}"\n'.format(self.text))
        f.write('description: "{}"\n'.format(self.comment))
        f.write('id: "{}"\n'.format(self.id))
        f.write('date: "{}"\n'.format(self.timestamp))
        f.write('featured: true\n')
        f.write('draft: false\n')
        f.write('author: "{}"\n'.format(self.email.split("@")[0]))
        f.write('tags: [{}]\n'.format(self.areas))
        f.write('categories: [{}]\n'.format(self.code))
        f.write('chapter: "{}"\n'.format(self.chapter))
        f.write('article: "{}"\n'.format(self.article))
        f.write('division: "{}"\n'.format(self.division))
        f.write('page: "{}"\n'.format(self.page_number))
        f.write('pageNumber: {}\n'.format(self.page_number))
        f.write('indexNumber: {}\n'.format(self.page_number))
        f.write('index: "{}"\n'.format(self.footnote_index))
        f.write('codelink: "{}"\n'.format("/pdf/annotated-code.pdf#page={}".format(self.page_number)))
        f.write("---\n")
        f.close() 

# annotation = Annotation(timestamp="2019-09-13T13:37:36.000Z", email="kevin.cmalughlin70@gmail.com", page_number=233, text="(9.4×Number of Units×Residents per Unit)/1,000=Acres of Parkland", 
#                           comment="Would this apply to site-plan lites?", 
#                           code="Error", 
#                           areas=["Missing Middle", "Parkland"],
#                             url="https://htmlpreview.github.io/?https://github.com/kmclaugh/land-development-code-review/blob/master/all_annotation.html")
# annotation.find_note_location(annotated_doc)


In [69]:
## Connect to Google Sheet
## Connect to our service account
scope = ['https://spreadsheets.google.com/feeds']
credentials = ServiceAccountCredentials.from_json_keyfile_name('./Jupyter_and_Google_Sheets.json', scope)
gc = gspread.authorize(credentials)
spreadsheet_key = '1pmnxdv0nNNGFEhRCPFaWGHlT6Oh8X6taYS3cgnM29y4'
book = gc.open_by_key(spreadsheet_key)
worksheet = book.worksheet("Form Responses 1")
table = worksheet.get_all_values()

##Convert table data into a dataframe
cdf = pd.DataFrame(table[1:], columns=table[0])

##Convert number strings to floats and ints
#cdf = cdf.apply(pd.to_numeric, errors='ignore')

##Convert date strings to datetime format
#cdf['Timestamp'] = pd.to_datetime(cdf['Timestamp'],infer_datetime_format=True)

##Drop no timestamps
cdf = cdf[pd.notnull(cdf['Timestamp'])]


In [70]:
cdf

,Timestamp,Email Address,Page Number,Text,Comment,Comment Codes,Tags
0,2019-10-04T17:46:33.000Z,kevin.mclaughlin70@gmail.com,1327,"Except as otherwise provided in this article, ...",Test,Question,ADUs


In [71]:
cdf["Annotation"] = cdf.apply(lambda x: Annotation(timestamp=x["Timestamp"], email=x["Email Address"], page_number=x["Page Number"], text=x["Text"], comment=x["Comment"], code=x["Comment Codes"], areas=x["Tags"]), axis=1)
cdf["Footnote Location"] = cdf.apply(lambda x: x["Annotation"].find_note_location(annotated_doc), axis=1)
cdf["Footnote Locationx"] = cdf.apply(lambda x: x["Footnote Location"][0], axis=1)
cdf["Footnote Locationy"] = cdf.apply(lambda x: x["Footnote Location"][1], axis=1)

cdf = cdf.sort_values(["Page Number", "Footnote Locationy", "Footnote Locationx"], ascending=[True, True, False])


In [72]:
## Find footnote indexes
current_index = 0
current_page = cdf.iloc[0]["Page Number"]
for index, row in cdf.iterrows():
    this_page = row["Page Number"]
    if current_page == this_page:
        current_index += 1
    else:
        current_page = this_page
        current_index = 1
    row["Annotation"].footnote_index = current_index
cdf["Footnote Index"] = cdf["Annotation"].map(lambda x: x.footnote_index)
cdf.head()

,Timestamp,Email Address,Page Number,Text,Comment,Comment Codes,Tags,Annotation,Footnote Location,Footnote Locationx,Footnote Locationy,Footnote Index
0,2019-10-04T17:46:33.000Z,kevin.mclaughlin70@gmail.com,1327,"Except as otherwise provided in this article, ...",Test,Question,ADUs,1327 - 1,"(482.975036621, 300.368438721)",482.975037,300.368439,1


In [73]:
## Update location for page level comments
cdf["Footnote Location"] = cdf.apply(lambda x: x["Annotation"].set_page_level_location(), axis=1)
cdf["Footnote Locationx"] = cdf.apply(lambda x: x["Footnote Location"][0], axis=1)
cdf["Footnote Locationy"] = cdf.apply(lambda x: x["Footnote Location"][1], axis=1)

In [74]:
cdf.apply(lambda x: x["Annotation"].add_to_pdf(annotated_doc), axis=1)
annotated_doc.save(annotated_doc.name, incremental=True, encryption=0)

In [75]:
site_file = "/Users/kevinmclaughlin/projects/code-commenting-site/static/pdf/annotated-code.pdf"
copyfile(annotated_doc.name, site_file)

In [76]:
cdf

,Timestamp,Email Address,Page Number,Text,Comment,Comment Codes,Tags,Annotation,Footnote Location,Footnote Locationx,Footnote Locationy,Footnote Index
0,2019-10-04T17:46:33.000Z,kevin.mclaughlin70@gmail.com,1327,"Except as otherwise provided in this article, ...",Test,Question,ADUs,1327 - 1,"(482.975036621, 300.368438721)",482.975037,300.368439,1


In [77]:
annotated_doc[0].rect.x1/2

306.0

## Create hugo files

In [78]:
##  Get section breakdown
## Connect to our service account
scope = ['https://spreadsheets.google.com/feeds']
credentials = ServiceAccountCredentials.from_json_keyfile_name('./Jupyter_and_Google_Sheets.json', scope)
gc = gspread.authorize(credentials)
spreadsheet_key = '1f7ZUpDVnW-FTdBrRL9D9qFgPZfFB-HrnBuiCQnZ2e1U'
book = gc.open_by_key(spreadsheet_key)
worksheet = book.worksheet("table of contents breakdown")
table = worksheet.get_all_values()

##Convert table data into a dataframe
sdf = pd.DataFrame(table[1:], columns=table[0])

##Convert number strings to floats and ints
sdf = sdf.apply(pd.to_numeric, errors='ignore')

In [79]:
sdf.head()

,,Level Index,Level Name,Level Value,Title,Key,Page,name,chapter,article,division,link,Last Page,Page Count
0,5,1,chapter,1,Chapter 23-1: Introduction,23-1,13,Introduction,1,nan,nan,https://ldc.aura-atx.org/pdf/plain-code.pdf#pa...,51,38
1,6,2,article,A,Article 23-1A: General Provisions,23-1A,17,General Provisions,1,A,nan,https://ldc.aura-atx.org/pdf/plain-code.pdf#pa...,33,16
2,7,3,division,1,"Division 23-1A-1: Title, Purpose, and Scope",23-1A-1,19,"Title, Purpose, and Scope",1,A,1,https://ldc.aura-atx.org/pdf/plain-code.pdf#pa...,23,4
3,8,3,division,2,Division 23-1A-2: Authority,23-1A-2,23,Authority,1,A,2,https://ldc.aura-atx.org/pdf/plain-code.pdf#pa...,25,2
4,9,3,division,3,Division 23-1A-3: Classification of Applicatio...,23-1A-3,25,Classification of Applications and Decisions,1,A,3,https://ldc.aura-atx.org/pdf/plain-code.pdf#pa...,29,4


In [80]:
def find_annotation_sections(annotation):
    chapter = sdf[(sdf["Level Name"] == "chapter") & (sdf["Page"] <= annotation.page_number) & (sdf["Last Page"] >= annotation.page_number)]
    print chapter
    if chapter.shape[0] > 0:
        annotation.chapter = chapter.iloc[0]["Level Value"]
        
    article = sdf[(sdf["Level Name"] == "article") & (sdf["Page"] <= annotation.page_number) & (sdf["Last Page"] >= annotation.page_number)]
    if article.shape[0] > 0:
        annotation.article = article.iloc[0]["Level Value"]
    
    division = sdf[(sdf["Level Name"] == "division") & (sdf["Page"] <= annotation.page_number) & (sdf["Last Page"] >= annotation.page_number)]
    if division.shape[0] > 0:
        annotation.division = division.iloc[0]["Level Value"]

    return annotation.chapter, annotation.article, annotation.division
cdf["Annotation"].apply(lambda x: find_annotation_sections(x))

0    (11, A, 2)
Name: Annotation, dtype: object

In [81]:
## Create section arrays
loc = "/Users/kevinmclaughlin/projects/code-commenting-site/data/"
chapter_json = sdf[(sdf["Level Name"] == "chapter")].set_index('Key').to_json(loc+"chapterdata.json", orient="index")
article_json = sdf[(sdf["Level Name"] == "article")].set_index('Key').to_json(loc+"articledata.json", orient="index")
division_json = sdf[(sdf["Level Name"] == "division")].set_index('Key').to_json(loc+"divisiondata.json", orient="index")

In [82]:
import sys
reload(sys)
sys.setdefaultencoding("utf-8")

loc = "/Users/kevinmclaughlin/projects/code-commenting-site/content/comments/"
cdf["Annotation"].apply(lambda x: x.create_hugo_file(loc))

reload(sys)
sys.setdefaultencoding("ASCII")